In [1]:
from intuitlib.client import AuthClient
from quickbooks import QuickBooks
from quickbooks.objects.account import Account
import pandas as pd

In [2]:
auth_client = AuthClient(
        client_id='ABVlDV6VbSq3SyfN6ShES7gauQ8LExHxnhuoDVsKaEOe3GmBrT',
        client_secret='zqQX950hO9CICCreDvfAvk3mE7tuJJKrGyyrRxEU',
        access_token=None,  # If you do not pass this in, the Quickbooks client will call refresh and get a new access token. 
        environment='production',
        redirect_uri='https://developer.intuit.com/v2/OAuth2Playground/RedirectUrl',
    )

In [3]:
client = QuickBooks(
        auth_client=auth_client,
        refresh_token='AB116945475244BlQAloOY4qPsm9zaekN4JTqKwHN5H60rL7dH',
        company_id='514655250',
    )

In [4]:
json_resp = client.get_report("ProfitAndLossDetail", {"start_date":"2023-01-01", "end_date":"2023-01-31"})

In [5]:
json_resp.keys()

dict_keys(['Header', 'Columns', 'Rows'])

In [6]:
json_resp['Columns']

{'Column': [{'ColTitle': 'Date', 'ColType': 'tx_date'},
  {'ColTitle': 'Transaction Type', 'ColType': 'txn_type'},
  {'ColTitle': 'Num', 'ColType': 'doc_num'},
  {'ColTitle': 'Name', 'ColType': 'name'},
  {'ColTitle': 'Memo/Description', 'ColType': 'memo'},
  {'ColTitle': 'Split', 'ColType': 'split_acc'},
  {'ColTitle': 'Amount', 'ColType': 'subt_nat_amount'},
  {'ColTitle': 'Balance', 'ColType': 'rbal_nat_amount'}]}

In [7]:
cols = [i["ColTitle"] for i in json_resp['Columns']['Column']]
report_info = json_resp['Header']

In [8]:
report_info

{'Time': '2023-06-23T15:07:59-07:00',
 'ReportName': 'ProfitAndLossDetail',
 'ReportBasis': 'Cash',
 'StartPeriod': '2023-01-01',
 'EndPeriod': '2023-01-31',
 'Currency': 'USD',
 'Option': [{'Name': 'NoReportData', 'Value': 'false'}]}

In [14]:
json_resp['Rows']['Row']

[{'Header': {'ColData': [{'value': 'Ordinary Income/Expenses'},
    {'value': ''},
    {'value': ''},
    {'value': ''},
    {'value': ''},
    {'value': ''},
    {'value': ''},
    {'value': ''}]},
  'Rows': {'Row': [{'Header': {'ColData': [{'value': 'Income'},
       {'value': ''},
       {'value': ''},
       {'value': ''},
       {'value': ''},
       {'value': ''},
       {'value': ''},
       {'value': ''}]},
     'Rows': {'Row': [{'Header': {'ColData': [{'value': 'Interest Earned',
           'id': '188'},
          {'value': ''},
          {'value': ''},
          {'value': ''},
          {'value': ''},
          {'value': ''},
          {'value': ''},
          {'value': ''}]},
        'Rows': {'Row': [{'ColData': [{'value': '2023-01-31'},
            {'value': 'Deposit', 'id': '3491'},
            {'value': ''},
            {'value': '', 'id': ''},
            {'value': 'Deposit Dividend 0.050% APY Earn Dividend, Deposit, Processed'},
            {'value': 'ENT Savings (Keyst

In [ ]:
get header_info

In [42]:
row_list = []
def proc_rows(rows, category = "", level=0):
    row_count = 0 # actual values row count
    for row in rows:
        if "Header" in row:
            header_col = row['Header']['ColData'][0]['value']
            if category == "":
                current_category = header_col
            else:
                current_category = f"{category}:{header_col}"
            print(current_category)
            proc_rows(row['Rows']['Row'], category = current_category, level = level+1)
        else:
            row_count+=1
            col_data = row['ColData']
            if len(col_data)==len(cols):
                cur_row = {cols[i]:col_data[i]['value'] for i in range(len(cols))}
                cur_row.update({"category":category})
                row_list.append(cur_row)
                print(f"row {row_count}")
            #parent_category = "_".join(parent_category.split("_")[:level-1])
        

In [43]:
proc_rows(json_resp["Rows"]["Row"][0]['Rows']['Row'])

Income
Income:Interest Earned
row 1
Income:Tithe
Income:Tithe:General Tithe
row 1
row 2
row 3
row 4
row 5
row 6
row 7
row 8
row 9
row 10
row 11
row 12
row 13
Expenses
Expenses:Facilities
Expenses:Facilities:Building/Office
Expenses:Facilities:Building/Office:Church Insurance
row 1
Expenses:Facilities:Building/Office:Communications
row 1
Expenses:Facilities:Building/Office:Office Supplies
row 1
Expenses:Facilities:Building/Office:Repair and Maintenance
row 1
row 2
row 3
Expenses:Facilities:Building/Office:Updates/Renovations
row 1
Expenses:Facilities:Government/Professional Fees
Expenses:Facilities:Government/Professional Fees:City Watershed Fees
row 1
Expenses:Facilities:Government/Professional Fees:Cleaning Service
row 1
Expenses:Facilities:Government/Professional Fees:Incorporation Fees
row 1
row 2
Expenses:Facilities:Government/Professional Fees:Utilities
row 1
Expenses:Facilities:Hospitality
Expenses:Facilities:Hospitality:Kitchen/Cleaning/Paper Supplies
row 1
Expenses:Facilities:S

In [33]:
pd.DataFrame(row_list)

,Date,Transaction Type,Num,Name,Memo/Description,Split,Amount,Balance,category
0,2023-01-31,Deposit,,,"Deposit Dividend 0.050% APY Earn Dividend, Dep...",ENT Savings (Keystone),.40,.40,Interest Earned
1,2023-01-03,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),145.35,145.35,General Tithe
2,2023-01-08,Deposit,,Larry Metzler,Metzler direct deposit,1002 ENT Checking (Keystone),1280.00,1425.35,General Tithe
3,2023-01-09,Deposit,,,"Deposit Bulk Checks 1,30 Deposit, Proc...",1002 ENT Checking (Keystone),1300.00,2725.35,General Tithe
4,2023-01-09,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),155.00,2880.35,General Tithe
5,2023-01-11,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),48.25,2928.60,General Tithe
6,2023-01-15,Deposit,,,"RDC Deposit RDC Item Count 1 Check, Deposit,...",1002 ENT Checking (Keystone),112.28,3040.88,General Tithe
7,2023-01-15,Deposit,,Larry Metzler,Metzler direct deposit,1002 ENT Checking (Keystone),250.00,3290.88,General Tithe
8,2023-01-18,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),38.31,3329.19,General Tithe
9,2023-01-19,Deposit,,,"Tithe.ly : Tithe.ly : 1800948 ACH, Deposit,...",1002 ENT Checking (Keystone),193.90,3523.09,General Tithe
